# Estimación de Word Vectors

Todos los métodos para obtener una representación del significado de una palabra utilizan como punto de partida un vocabulario $V = \{ casa, árbol, él, el, la, ellos, ellas, \ldots \}$ finito.

## Métodos por PCA

Explicar.

## Algoritmos `word2vec`

> **Resumen:** El objetivo acá es obtener una representación del significado de las palabras como producto secundario de un modelo de aprendizaje supervisado. Es decir, se va a diseñar un clasificador que permita predecir una palabra a partir de otra (ambas pertenecientes al vocabulario $V$) y se van a utilizar los parámetros del modelo como representación del significado de la palabra. 

Dado un vocabulario $V$, de tamaño $|V|$, se define que

$$
h_j = 
\begin{bmatrix}
0 \\
\vdots \\
0 \\
1 \\
0 \\
\vdots \\
0
\end{bmatrix}
\in \mathbb{R}^{|V|}
$$

es un vector *one-hot* con su j-ésima coordenada igual a 1, y cero en otro lado. También definimos las variables aleatorias $\mathbf{w}$ y $\mathbf{c}$ con realizaciones en $\mathbb{R}^{|V|}$ que representan la ocurrencia de una palabra central y de su contexto, respectivamente, a través de su correspondiente vector *one-hot*. Por ejemplo, para un vocabulario $V=\{ casa, conjunto, el, diez, vacío, la, ellos, ellas, \ldots \}$, las palabras "conjunto" y "vacío" se representan por los vectores $h_2$ y $h_5$, respectivamente. Además, la frase "conjunto vacío" puede armarse de dos maneras posibles: 
1. "conjunto" es la palabra central y "vacío" es su contexto (en cuyo caso $\mathbf{w}$ habrá adoptado el valor $h_2$ y $\mathbf{c}$ el valor $h_5$), o bien
2. "vacío" es la palabra central y "conjunto" es su contexto (en cuyo caso $\mathbf{w}$ habrá adoptado el valor $h_5$ y $\mathbf{c}$ el valor $h_2$).
Para el caso en que se tiene una frase compuesta por más de dos palabras, se suele tomar el promedio de los vectores *one-hot* que conforman la frase para representar el contexto o la palabra central. Por ejemplo, para el caso de la frase "el conjunto vacío" en que se toma a "conjunto" como palabra central y a $\{"\!el\!", "\!vacío\!"\}$ como contexto, se podrá considerar que las variables $\mathbf{w}$ y $\mathbf{c}$ adoptaron los valores $h_2$ y $\frac{1}{2}(h_3 + h_5)$ respectivamente.

Con estas definiciones, podemos formalizar el funcionamiento básico de los algoritmos `word2vec`. Dado un vocabulario $V$ y un corpus de texto (cuyas palabras estén contenidas en $V$), se buscará estimar la probabilidad de que, dado un contexto $c$ se obtenga una palabra central $w$:

$$
P(\mathbf{w}=w | \mathbf{c}=c)
$$

Con esta probabilidad, puede definirse un clasificador bayesiano que asigne una palabra $w$ a un determinado contexto $c$ y tomar como medida de la performance el error de predicciones acertadas:

![alt text]("word2vec.png")

Ahora pasamos a estimar la probabilidad $P(w|c)$. Para un problema de clasificación en el cual se estima la probabilidad de cada clase dado el vector de *features* por una softmax, normalmente se tiene que

$$
\begin{equation}
\begin{split}
P(y|x;W) &= softmax(Wx) \\[.5em]
&= \frac{e^{Wx}}{\sum_{i=1}^{K}e^{w_i^Tx}} \\[.5em]
&=
\begin{bmatrix}
P(y=1|x) \\
\vdots \\
P(y=K|x)
\end{bmatrix}
\end{split}
\end{equation}
$$

con

$$
W = \begin{bmatrix}
- w_1^T - \\
\vdots \\
- w_K^T - \\
\end{bmatrix}
$$

y la estimación por *maximum likelihood* resulta en encontrar la matriz $W$ que minimice la función

$$
\begin{equation}
\begin{split}
NLL(W) &= -log(l(W)) = -log\left( \prod_{i=1}^N P(y^{(i)}|x^{(i)};W)\right)\\[.5em]
&= - \sum_{i=1}^N log\left(P(y^{(i)}|x^{(i)};W)\right)\\[.5em]
\end{split}
\end{equation}
$$

Con el fin de obtener un algoritmo iterativo, se puede encontrar el gradiente de $NLL(W)$ e iterar hasta llegar al mínimo:

$$
W_{new} \leftarrow W_{old} - \mu \nabla_W NLL(W)
$$

que para el caso de la softmax, el gradiente tiene la forma

$$
\nabla_W NLL(W) = 
\begin{bmatrix}
| & & | \\
\sum_{n=1}^N x^{(n)} \left(softmax(w_{y^{(n)}}^T x^{(n)}) - \mathbb{1}_{\{y^{(n)}=1\}}\right) & \ldots & \sum_{n=1}^N x^{(n)} \left(softmax(w_{y^{(n)}}^T x^{(n)}) - \mathbb{1}_{\{y^{(n)}=K\}}\right) \\
| & & | \\
\end{bmatrix}
$$

$$
\newcommand{\U}{\mathcal{U}}
\newcommand{\V}{\mathcal{V}}
$$


En este caso, se va a estimar la probabilidad a posteriori mediante una softmax, pero se va a tomar $W=\U\V$, donde 

$$
\begin{align}
\U = 
\begin{bmatrix}
- \;u_1\; - \\
\vdots \\
- \;u_{|V|}\; - 
\end{bmatrix} \in \mathbb{R}^{|V|\times n} & \hspace{2em} &
\V = 
\begin{bmatrix}
| & & | \\
v_1 & \ldots & v_{|V|} \\
| & & | \\
\end{bmatrix} \in \mathbb{R}^{n\times |V|}
\end{align}
$$